# LCC - Matrix Approach

This notebook demonstrates our approach to life cycle costing (LCC), which can handle reference products with a price of zero or missing price data.

We adapt the example from Heijungs et al. (2013) and Moreau et al. (2015), with the addition of a timber production step. In our case, the price of wood is assumed to be unknown. As a result, we must address two challenges: a functional unit with a monetary value of zero, and a production activity where the product has known costs but an unknown market price.

Authors:<br>
  - Alvaro J. Hahn Menacho (alvaro.hahn-menacho@psi.ch)<br>
  - Romain Sacchi (romain.sacchi@psi.ch)

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

| **Activity**              | **Reference product**  | **Physical inputs**                               | **Physical outputs**              | **Price of the reference product** |
|---------------------------|------------------------|---------------------------------------------------|-----------------------------------|-------------------------------------|
| Production of electricity | Electricity            | -                                                 | 1 MJ electricity                  | 5  \$/MJ                             |
| Production of wood        | Wood                   | • 0.5 MJ electricity<br>• 7 kg timber              | 5 kg wood                         | N.A.                               |
| Production of timber      | Timber                 | -                                                 | 1 kg timber                       | 1  \$/kg                             |
| Production of chair       | Chair                  | • 2 MJ electricity<br>• 5 kg wood                  | 5 pieces chair                    | 25  \$/piece                         |
| Use of chair              | Sitting                | • 5 pieces chair<br>• 5 pieces of broken chair     | 10 years of sitting               | 0  \$/year                           |
| Disposal of broken chair  | Broken chair           | -                                                 | -1 piece chair                    | -2  \$/piece                         |


## a) Technology matrix, **A**

Firstly, let's define the row and column labels

In [2]:
rows_A = ["Electricity", "Wood", "Timber", "Chair", "Sitting", "Broken chair"]

In [3]:
columns_A = [
    "Production of electricity", 
    "Production of wood", 
    "Production of timber", 
    "Production of chair", 
    "Use of chair", 
    "Disposal of broken chair",
]

Populating the A matrix:

In [4]:
A_data = [
    [1, -0.5, 0, -2, 0, 0],
    [0, 5, 0, -5, 0, 0],
    [0, -7, 1, 0, 0, 0],
    [0, 0, 0, 5, -5, 0],
    [0, 0, 0, 0, 10, 0],
    [0, 0, 0, 0, 5, -1]
]

Combine everything in a dataframe:

In [5]:
A_matrix = pd.DataFrame(A_data, index=rows_A, columns=columns_A)
A_matrix

,Production of electricity,Production of wood,Production of timber,Production of chair,Use of chair,Disposal of broken chair
Electricity,1,-0.5,0,-2,0,0
Wood,0,5.0,0,-5,0,0
Timber,0,-7.0,1,0,0,0
Chair,0,0.0,0,5,-5,0
Sitting,0,0.0,0,0,10,0
Broken chair,0,0.0,0,0,5,-1


## a) Price vector, **α**

Again, let's start defining the rows

In [6]:
rows_alpha = ["Electricity", "Wood", "Timber", "Chair", "Sitting", "Broken chair"]

Populating the B matrix

In [7]:
alpha_data = [
    [5],
    [0],
    [1],
    [25],
    [0],
    [-2],
]

In [8]:
alpha_matrix = pd.DataFrame(alpha_data, index=rows_alpha, columns=["Price"])
alpha_matrix

,Price
Electricity,5
Wood,0
Timber,1
Chair,25
Sitting,0
Broken chair,-2


Aligned with the work by Moreau et al. (2015), we do not want activities witth negative value added. Thus, the zero price of the refernce product and the resulting negative value added of the use activity is considered a conceptual error. Furthermore, we want to eliminate cutoff errors by inferring missing costs.

Hence, we modify the set of prices so that:

1. Each product’s price covers the cost of its inputs, propagated through the system.<br><br>
Additionally:<br><br>
3. Anchored prices are respected as lower bounds.
4. The solution yields minimum total cost, consistent with the system structure.

## *Formulation for consistent price vector in Life Cycle Costing

### **Input**


- Technology matrix \( $A$ \):

$$
A \in \mathbb{R}^{n \times n},\quad \text{where}\quad A_{ij}\text{ is the amount of product }i\text{ used (negative) or produced (positive) by process }j.
$$

- Original price vector \( $p^{0}$ \):

$$
p^{0} \in \mathbb{R}^{n},\quad\text{possibly incomplete or inconsistent.}
$$

- Set of indices with anchored prices \( $\mathcal{K}$ \):

$$
\mathcal{K} \subseteq \{1,\dots,n\}
$$

- Anchor prices \( $a_{k}$ \):

$$
a_{k} \in \mathbb{R}_{>0},\quad\text{lower bound (anchor) price for each } k \in \mathcal{K}.
$$


### **Step 1: Normalize A to Input Coefficients Matrix A∗A∗**

Let each column $j$ of $A$ represent a production process with output $A_{jj} > 0$.

Define the normalized input coefficients $A_{ij}^*$ as:

$$
A_{ij}^* =
\begin{cases}
\displaystyle -\frac{A_{ij}}{A_{jj}} & \text{if } i \ne j \text{ and } A_{jj} \ne 0 \\
0 & \text{otherwise}
\end{cases}
$$

This gives the amount of input $i$ required per unit of output from process $j$.

### **Step 2: Pricing Constraints**

We impose that for every product $j \in \{1, \dots, n\}$ that has at least one input:

$$
p_j \geq \sum_{i \ne j} A_{ij}^* \cdot p_i
$$

This ensures that the price of product $j$ is at least equal to the sum of the prices of the inputs required to produce one unit of it.

We define three types of pricing constraints:

- For every product $k \in \mathcal{F}$ that has no inputs (i.e. independent processes), we fix its price to the value provided in the original price vector:

$$
p_k = a_k
$$

- For every product $k \in \mathcal{K}$ with a known positive price and at least one input, we enforce a lower bound:

$$
p_k \geq a_k
$$

- For all other products, we require:

$$
p_j \geq 0
$$

**Treatment activities** are handled consistently through this rule:  
If a treatment process has no inputs and a non-positive price in the original vector, that price is fixed. This avoids assigning positive value to waste treatment and ensures disposal costs propagate upstream.

### **Step 3: Objective Function**

To make the solution conservative and efficient, we minimize the total sum of prices:

$$
\min_{p \in \mathbb{R}^n} \sum_{j=1}^{n} p_j
$$

This selects the lowest-cost solution consistent with the production structure and pricing constraints.

### **Final Linear Program**

Minimize:

$$
\sum_{j=1}^{n} p_j
$$

Subject to:

$$
p_j - \sum_{i \ne j} A_{ij}^* \cdot p_i \geq 0 \quad \forall j \notin \mathcal{F}
$$

$$
p_k = a_k \quad \forall k \in \mathcal{F}
$$

$$
p_k \geq a_k \quad \forall k \in \mathcal{K}
$$

$$
p_j \geq 0 \quad \forall j \in \{1, \dots, n\}
$$

In [9]:
def compute_consistent_alpha(A_matrix: pd.DataFrame,
                             alpha_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Compute a consistent alpha (price) vector using constrained Leontief-style cost propagation.
    
    Rules:
    - If a process has no inputs, its price is fixed to the value given in alpha_matrix (positive or negative).
    - If a process has inputs and a positive price, the price is enforced as a lower bound.
    - All other prices are inferred, but must be non-negative.
    
    Parameters:
    - A_matrix (pd.DataFrame): Square technology matrix.
    - alpha_matrix (pd.DataFrame): Initial price vector (some values may be zero or negative).
    
    Returns:
    - pd.DataFrame: New consistent price vector.
    """
    rows = list(A_matrix.index)
    n = len(rows)

    T = A_matrix.to_numpy()
    A_star = np.zeros_like(T)
    for j in range(n):
        output = T[j, j]
        if output == 0:
            continue
        for i in range(n):
            if i != j:
                A_star[i, j] = -T[i, j] / output

    original_prices = alpha_matrix["Price"].values.astype(float)
    bounds = []
    A_ub = []
    b_ub = []

    for j in range(n):
        column = T[:, j]
        is_independent = np.allclose(column - np.eye(n)[:, j] * T[j, j], 0)
        price = original_prices[j]

        if is_independent:
            bounds.append((price, price))  # Fix price
            continue  # Don't add constraint: p_j ≥ inputs (inputs are none)
        else:
            # Constraint: p_j ≥ sum of inputs
            row = np.zeros(n)
            row[j] = -1
            for i in range(n):
                row[i] += A_star[i, j]
            A_ub.append(row)
            b_ub.append(0)

            # Set bounds
            if price > 0:
                bounds.append((price, None))
            else:
                bounds.append((0, None))

    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)
    c = np.ones(n)

    result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

    if result.success:
        return pd.DataFrame({"Price": result.x}, index=rows)
    else:
        raise RuntimeError("Linear program failed to find a consistent price vector.")

In [10]:
new_alpha = compute_consistent_alpha(A_matrix, alpha_matrix)
print(new_alpha)

              Price
Electricity     5.0
Wood            1.9
Timber          1.0
Chair          25.0
Sitting        13.5
Broken chair   -2.0


## b) Demand vector, **f**

In [11]:
f_data = [
    [0],
    [0],
    [0],
    [0],
    [1],
    [0],
]

In [12]:
f_vector = pd.DataFrame(f_data, index=rows_A, columns=["Final demand"])
f_vector

,Final demand
Electricity,0
Wood,0
Timber,0
Chair,0
Sitting,1
Broken chair,0


## c) Scaling factors

Scaling factors $s$ are obtained by solving the standard linear system:

$$
s = A^{-1} \cdot f
$$

where:
- $A$ is the technology matrix,
- $f$ is the final demand vector,
- and $s$ is the scaling vector representing the activity levels required to meet $f$.

In [13]:
s = np.linalg.solve(A_matrix.to_numpy(), f_vector.to_numpy())
s

array([[0.25],
       [0.1 ],
       [0.7 ],
       [0.1 ],
       [0.1 ],
       [0.5 ]])

And the process matrix:

In [14]:
A_scaled = A_matrix.to_numpy() * s 
A_scaled

array([[ 0.25 , -0.125,  0.   , -0.5  ,  0.   ,  0.   ],
       [ 0.   ,  0.5  ,  0.   , -0.5  ,  0.   ,  0.   ],
       [ 0.   , -4.9  ,  0.7  ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.5  , -0.5  ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  2.5  , -0.5  ]])

## d) Transform the 'physical' technology matrix into its monetary form

The monetary technology matrix is obtained by scaling each **row** of the physical technology matrix $A$ by the corresponding price in the vector $\alpha$:

$$
A_m = \text{diag}(\alpha) \cdot A
$$

where:
- $\alpha$ is the price vector (in units of currency per unit of product),
- $A$ is the physical technology matrix (product × process),
- and $A_m$ is the monetary technology matrix (currency × process),
- i.e., each entry of $A_m$ represents the cost contribution of product $i$ to process $j$.

In [15]:
Am = np.diag(new_alpha.to_numpy().flatten()) @ A_matrix.to_numpy()
Am_df = pd.DataFrame(Am, index=A_matrix.index, columns=A_matrix.columns)
Am_df

,Production of electricity,Production of wood,Production of timber,Production of chair,Use of chair,Disposal of broken chair
Electricity,5.0,-2.5,0.0,-10.0,0.0,0.0
Wood,0.0,9.5,0.0,-9.5,0.0,0.0
Timber,0.0,-7.0,1.0,0.0,0.0,0.0
Chair,0.0,0.0,0.0,125.0,-125.0,0.0
Sitting,0.0,0.0,0.0,0.0,135.0,0.0
Broken chair,0.0,0.0,0.0,0.0,-10.0,2.0


## e) Scale the monetary technology matrix

Each column of the monetary technology matrix $A_m$ is scaled by the corresponding activity level in $s_m$:

$$
A_{m, \text{scaled}} = A_m \cdot \text{diag}(s_m)
$$

This yields the scaled monetary process matrix, where each entry represents the actual monetary flow associated with product $i$ and process $j$.

In [16]:
f_m = pd.Series(0.0, index=A_matrix.index)
f_m["Sitting"] = new_alpha.loc["Sitting", "Price"]
f_m

Electricity      0.0
Wood             0.0
Timber           0.0
Chair            0.0
Sitting         13.5
Broken chair     0.0
dtype: float64

In [17]:
Am = Am_df.to_numpy()
s_m = np.linalg.solve(Am, f_m.to_numpy())

In [18]:
Am_scaled = Am * s_m
Am_scaled_df = pd.DataFrame(Am_scaled, index=A_matrix.index, columns=A_matrix.columns)
Am_scaled_df

,Production of electricity,Production of wood,Production of timber,Production of chair,Use of chair,Disposal of broken chair
Electricity,1.25,-0.25,0.0,-1.00,0.0,0.0
Wood,0.00,0.95,0.0,-0.95,0.0,0.0
Timber,0.00,-0.70,0.7,0.00,0.0,0.0
Chair,0.00,0.00,0.0,12.50,-12.5,0.0
Sitting,0.00,0.00,0.0,0.00,13.5,0.0
Broken chair,0.00,0.00,0.0,0.00,-1.0,1.0


## f) Value added per process, $v$


The total cost (value added) per process is obtained by summing all monetary inputs across rows:

$$
v = A_{m, \text{scaled}}^\top \cdot \mathbf{l}
$$

where:
- $A_{m, \text{scaled}}^\top$ is the transposed scaled monetary matrix,
- $\mathbf{l}$ is a vector of ones (i.e., the summation operator),
- and $v$ is a vector containing the total cost of each process.

In [19]:
l = np.ones(Am_scaled_df.shape[0])
v = Am_scaled_df.T @ l 

In [20]:
value_added = pd.DataFrame({"Value added": v}, index=A_matrix.columns)
value_added

,Value added
Production of electricity,1.25
Production of wood,0.00
Production of timber,0.70
Production of chair,10.55
Use of chair,0.00
Disposal of broken chair,1.00


The sum of the $v$ vector is 13.5$, namely the life cycle cost. We thus define the life cycle cost as "the sum of all value added over the life cycle".